# Análisis de Datos para Reconocimiento de Actividades Humanas
## Usando el The ExtraSensory Dataset - A dataset for behavioral context recognition in-the-wild from mobile sensors
http://extrasensory.ucsd.edu/

Content of the primary data files:
There are 60 'csv.gz' files, one for each participant (user, subject) in the data collection.
Each of these files has filename with the form:[UUID].features_labels.csv.gz where each user has a unique (randomly generated) universal user identification (UUID) number.
Each file is a textual CSV file, compressed using the gzip format.

Within every user's CSV file:
-----------------------------
- The first row specifies the columns of the file.
- Every other row refers to an example from the user. The examples are sorted according to the primary key - the timestamp.
- The columns:

-- First column is 'timestamp'. This is represented as standard number of seconds since the epoch. Every example has its timestamp, indicating the minute when the example was recorded

-- Second, come columns for the extracted features.
   Unavailable features are represented with 'nan'.
   The name of each feature contains reference to the sensor it was extracted from, in the form [sensor_name]:[feature_name].
   The current version contains features from the following sensors, with sensor names:
--- raw_acc: Accelerometer from the phone. The 'raw' version of acceleration (as opposed to the decomposed versions of gravity and user-acceleration).   
--- proc_gyro: Gyroscope from the phone. Processed version of gyroscope measurements (the OS calculates a version that removes drift).
--- raw_magnet: Magnetometer from the phone. Raw version (as opposed to bias-fixed version that the OS also provides).
--- watch_acceleration: Accelerometer from the watch.
--- watch_heading: Heading from the compass on the watch.
--- location: Location services. These features were extracted offline for every example from the sequence of latitude-longitude-altitude updates from the example's minute.
              These features regard only to relative-location (not absolute location in the world) - meaning, they describe variability of movement within the minute.
--- location_quick_features: Location services. These features were calculated on the phone when data was collected. 
                             These are available even in cases that the other location features are not because the user wanted to conceal their absolute location coordinates.
							 These quick features are very simple heuristics that approximate the more thoughtful offline features.
--- audio_naive: Microphone. These naive features are simply averages and standard deviations of the 13 MFCCs from the ~20sec recording window of every example.
--- discrete: Phone-state. These are binary indicators for the state of the phone.
              Notice that time_of_day features are also considered phone-state features (also have prefix 'discrete:'), but their columns appear not right after the other 'discrete' columns.

In [1]:
import findspark
findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("HumanActivityETL")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.driver.memory", "15g")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [3]:
from pyspark.sql.functions import *;
from pyspark.sql.types import *;
from scipy.stats import *
import pandas as pd
from scipy import stats
from  pyspark.sql.functions import input_file_name

# Import all files from volume /dataset/features_labels

In [4]:
from functools import reduce
from pyspark.sql import DataFrame
import glob

path = r'/dataset/features_labels' # files in docker volume
all_files = glob.glob(path + "/*.csv.gz")

li = []

for filename in all_files:
    df_import = (
    spark.read.
    option("delimiter", ',').
    csv(filename,
        #schema=schema,
        header=True,
        ignoreLeadingWhiteSpace=True,
        ignoreTrailingWhiteSpace=True,
        nullValue='NA')
     )
    li.append(df_import)
    # Append all files imported in a single dataframe
    df = reduce(DataFrame.unionAll, li)

In [5]:
# Get user ID in a new column from each file name 

# Get file name in a new column
df = df.withColumn("filename", input_file_name())

# Get User ID from the filename column
df = df.withColumn("user_id", substring("filename", 33,36))

# Show values
df.select("user_id").show(n=5, vertical=True,truncate=100)

-RECORD 0---------------------------------------
 user_id | 797D145F-3858-4A7F-A7C2-A4EB721E133C 
-RECORD 1---------------------------------------
 user_id | 797D145F-3858-4A7F-A7C2-A4EB721E133C 
-RECORD 2---------------------------------------
 user_id | 797D145F-3858-4A7F-A7C2-A4EB721E133C 
-RECORD 3---------------------------------------
 user_id | 797D145F-3858-4A7F-A7C2-A4EB721E133C 
-RECORD 4---------------------------------------
 user_id | 797D145F-3858-4A7F-A7C2-A4EB721E133C 
only showing top 5 rows



In [6]:
# Remove column not required
df = df.drop(
"filename",
"raw_acc:magnitude_stats:percentile25",
"raw_acc:magnitude_stats:percentile50",
"raw_acc:magnitude_stats:percentile75",
"raw_acc:magnitude_spectrum:log_energy_band0",
"raw_acc:magnitude_spectrum:log_energy_band1",
"raw_acc:magnitude_spectrum:log_energy_band2",
"raw_acc:magnitude_spectrum:log_energy_band3",
"raw_acc:magnitude_spectrum:log_energy_band4",
"raw_acc:magnitude_autocorrelation:period",
"raw_acc:magnitude_autocorrelation:normalized_ac",
"proc_gyro:magnitude_stats:percentile25",
"proc_gyro:magnitude_stats:percentile50",
"proc_gyro:magnitude_stats:percentile75",
"proc_gyro:magnitude_spectrum:log_energy_band0",
"proc_gyro:magnitude_spectrum:log_energy_band1",
"proc_gyro:magnitude_spectrum:log_energy_band2",
"proc_gyro:magnitude_spectrum:log_energy_band3",
"proc_gyro:magnitude_spectrum:log_energy_band4",
"proc_gyro:magnitude_autocorrelation:period",
"proc_gyro:magnitude_autocorrelation:normalized_ac",
"raw_magnet:magnitude_stats:percentile25",
"raw_magnet:magnitude_stats:percentile50",
"raw_magnet:magnitude_stats:percentile75",
"raw_magnet:magnitude_spectrum:log_energy_band0",
"raw_magnet:magnitude_spectrum:log_energy_band1",
"raw_magnet:magnitude_spectrum:log_energy_band2",
"raw_magnet:magnitude_spectrum:log_energy_band3",
"raw_magnet:magnitude_spectrum:log_energy_band4",
"raw_magnet:magnitude_autocorrelation:period",
"raw_magnet:magnitude_autocorrelation:normalized_ac",
"raw_magnet:avr_cosine_similarity_lag_range0",
"raw_magnet:avr_cosine_similarity_lag_range1",
"raw_magnet:avr_cosine_similarity_lag_range2",
"raw_magnet:avr_cosine_similarity_lag_range3",
"raw_magnet:avr_cosine_similarity_lag_range4",
"watch_acceleration:magnitude_stats:percentile25",
"watch_acceleration:magnitude_stats:percentile50",
"watch_acceleration:magnitude_stats:percentile75",
"watch_acceleration:magnitude_spectrum:log_energy_band0",
"watch_acceleration:magnitude_spectrum:log_energy_band1",
"watch_acceleration:magnitude_spectrum:log_energy_band2",
"watch_acceleration:magnitude_spectrum:log_energy_band3",
"watch_acceleration:magnitude_spectrum:log_energy_band4",
"watch_acceleration:magnitude_autocorrelation:period",
"watch_acceleration:magnitude_autocorrelation:normalized_ac",
"watch_acceleration:spectrum:x_log_energy_band0",
"watch_acceleration:spectrum:x_log_energy_band1",
"watch_acceleration:spectrum:x_log_energy_band2",
"watch_acceleration:spectrum:x_log_energy_band3",
"watch_acceleration:spectrum:x_log_energy_band4",
"watch_acceleration:spectrum:y_log_energy_band0",
"watch_acceleration:spectrum:y_log_energy_band1",
"watch_acceleration:spectrum:y_log_energy_band2",
"watch_acceleration:spectrum:y_log_energy_band3",
"watch_acceleration:spectrum:y_log_energy_band4",
"watch_acceleration:spectrum:z_log_energy_band0",
"watch_acceleration:spectrum:z_log_energy_band1",
"watch_acceleration:spectrum:z_log_energy_band2",
"watch_acceleration:spectrum:z_log_energy_band3",
"watch_acceleration:spectrum:z_log_energy_band4",
"watch_acceleration:relative_directions:avr_cosine_similarity_lag_range0",
"watch_acceleration:relative_directions:avr_cosine_similarity_lag_range1",
"watch_acceleration:relative_directions:avr_cosine_similarity_lag_range2",
"watch_acceleration:relative_directions:avr_cosine_similarity_lag_range3",
"watch_acceleration:relative_directions:avr_cosine_similarity_lag_range4",
"raw_acc:magnitude_stats:moment3",
"raw_acc:magnitude_stats:moment4",
"proc_gyro:magnitude_stats:moment3",
"proc_gyro:magnitude_stats:moment4",
"raw_magnet:magnitude_stats:moment3",
"raw_magnet:magnitude_stats:moment4",
"watch_acceleration:magnitude_stats:moment3",
"watch_acceleration:magnitude_stats:moment4",
"watch_heading:mom3_cos",
"watch_heading:mom4_cos",
"watch_heading:mom3_sin",
"watch_heading:mom4_sin",
"audio_naive:mfcc0:mean",
"audio_naive:mfcc1:mean",
"audio_naive:mfcc2:mean",
"audio_naive:mfcc3:mean",
"audio_naive:mfcc4:mean",
"audio_naive:mfcc5:mean",
"audio_naive:mfcc6:mean",
"audio_naive:mfcc7:mean",
"audio_naive:mfcc8:mean",
"audio_naive:mfcc9:mean",
"audio_naive:mfcc10:mean",
"audio_naive:mfcc11:mean",
"audio_naive:mfcc12:mean",
"audio_naive:mfcc0:std",
"audio_naive:mfcc1:std",
"audio_naive:mfcc2:std",
"audio_naive:mfcc3:std",
"audio_naive:mfcc4:std",
"audio_naive:mfcc5:std",
"audio_naive:mfcc6:std",
"audio_naive:mfcc7:std",
"audio_naive:mfcc8:std",
"audio_naive:mfcc9:std",
"audio_naive:mfcc10:std",
"audio_naive:mfcc11:std",
"audio_naive:mfcc12:std",
"audio_properties:max_abs_value",
"audio_properties:normalization_multiplier",
"discrete:app_state:is_active",
"discrete:app_state:is_inactive",
"discrete:app_state:is_background",
"discrete:app_state:missing",
"discrete:battery_plugged:is_ac",
"discrete:battery_plugged:is_usb",
"discrete:battery_plugged:is_wireless",
"discrete:battery_plugged:missing",
"discrete:battery_state:is_unknown",
"discrete:battery_state:is_unplugged",
"discrete:battery_state:is_not_charging",
"discrete:battery_state:is_discharging",
"discrete:battery_state:is_charging",
"discrete:battery_state:is_full",
"discrete:battery_state:missing",
"discrete:on_the_phone:is_False",
"discrete:on_the_phone:is_True",
"discrete:on_the_phone:missing",
"discrete:ringer_mode:is_normal",
"discrete:ringer_mode:is_silent_no_vibrate",
"discrete:ringer_mode:is_silent_with_vibrate",
"discrete:ringer_mode:missing",
"discrete:wifi_status:is_not_reachable",
"discrete:wifi_status:is_reachable_via_wifi",
"discrete:wifi_status:is_reachable_via_wwan",
"discrete:wifi_status:missing",
"label_source"
)

In [7]:
# Rename column names

df = df.withColumnRenamed("timestamp","timestamp")  \
.withColumnRenamed("raw_acc:magnitude_stats:mean","acc_magnitude_mean")  \
.withColumnRenamed("raw_acc:magnitude_stats:std","acc_magnitude_std")  \
.withColumnRenamed("raw_acc:magnitude_stats:value_entropy","acc_magnitude_value_entropy")  \
.withColumnRenamed("raw_acc:magnitude_stats:time_entropy","acc_magnitude_time_entropy")  \
.withColumnRenamed("raw_acc:magnitude_spectrum:spectral_entropy","acc_spec_spectral_entropy")  \
.withColumnRenamed("raw_acc:3d:mean_x","acc_3d_mean_x")  \
.withColumnRenamed("raw_acc:3d:mean_y","acc_3d_mean_y")  \
.withColumnRenamed("raw_acc:3d:mean_z","acc_3d_mean_z")  \
.withColumnRenamed("raw_acc:3d:std_x","acc_3d_std_x")  \
.withColumnRenamed("raw_acc:3d:std_y","acc_3d_std_y")  \
.withColumnRenamed("raw_acc:3d:std_z","acc_3d_std_z")  \
.withColumnRenamed("raw_acc:3d:ro_xy","acc_3d_ro_x")  \
.withColumnRenamed("raw_acc:3d:ro_xz","acc_3d_ro_y")  \
.withColumnRenamed("raw_acc:3d:ro_yz","acc_3d_ro_z")  \
.withColumnRenamed("proc_gyro:magnitude_stats:mean","gyro_magnitude_mean")  \
.withColumnRenamed("proc_gyro:magnitude_stats:std","gyro_magnitude_std")  \
.withColumnRenamed("proc_gyro:magnitude_stats:value_entropy","gyro_magnitude_value_entropy")  \
.withColumnRenamed("proc_gyro:magnitude_stats:time_entropy","gyro_magnitude_time_entropy")  \
.withColumnRenamed("proc_gyro:magnitude_spectrum:spectral_entropy","gyro_spec_spectral_entropy")  \
.withColumnRenamed("proc_gyro:3d:mean_x","gyro_3d_mean_x")  \
.withColumnRenamed("proc_gyro:3d:mean_y","gyro_3d_mean_y")  \
.withColumnRenamed("proc_gyro:3d:mean_z","gyro_3d_mean_z")  \
.withColumnRenamed("proc_gyro:3d:std_x","gyro_3d_std_x")  \
.withColumnRenamed("proc_gyro:3d:std_y","gyro_3d_std_y")  \
.withColumnRenamed("proc_gyro:3d:std_z","gyro_3d_std_z")  \
.withColumnRenamed("proc_gyro:3d:ro_xy","gyro_3d_ro_xy")  \
.withColumnRenamed("proc_gyro:3d:ro_xz","gyro_3d_ro_xz")  \
.withColumnRenamed("proc_gyro:3d:ro_yz","gyro_3d_ro_yz")  \
.withColumnRenamed("raw_magnet:magnitude_stats:mean","magnet_magnitude_mean")  \
.withColumnRenamed("raw_magnet:magnitude_stats:std","magnet_magnitude_std")  \
.withColumnRenamed("raw_magnet:magnitude_stats:value_entropy","magnet_magnitude_value_entropy")  \
.withColumnRenamed("raw_magnet:magnitude_stats:time_entropy","magnet_magnitude_time_entropy")  \
.withColumnRenamed("raw_magnet:magnitude_spectrum:spectral_entropy","magnet_spec_spectral_entropy")  \
.withColumnRenamed("raw_magnet:3d:mean_x","magnet_3d_mean_x")  \
.withColumnRenamed("raw_magnet:3d:mean_y","magnet_3d_mean_y")  \
.withColumnRenamed("raw_magnet:3d:mean_z","magnet_3d_mean_z")  \
.withColumnRenamed("raw_magnet:3d:std_x","magnet_3d_std_x")  \
.withColumnRenamed("raw_magnet:3d:std_y","magnet_3d_std_y")  \
.withColumnRenamed("raw_magnet:3d:std_z","magnet_3d_std_z")  \
.withColumnRenamed("raw_magnet:3d:ro_xy","magnet_3d_ro_xy")  \
.withColumnRenamed("raw_magnet:3d:ro_xz","magnet_3d_ro_xz")  \
.withColumnRenamed("raw_magnet:3d:ro_yz","magnet_3d_ro_yz")  \
.withColumnRenamed("watch_acceleration:magnitude_stats:mean","acc_watch_magnitude_mean")  \
.withColumnRenamed("watch_acceleration:magnitude_stats:std","acc_watch_magnitude_std")  \
.withColumnRenamed("watch_acceleration:magnitude_stats:value_entropy","acc_watch_magnitude_value_entropy")  \
.withColumnRenamed("watch_acceleration:magnitude_stats:time_entropy","acc_watch_magnitude_time_entropy")  \
.withColumnRenamed("watch_acceleration:magnitude_spectrum:spectral_entropy","acc_watch_spec_spectral_entropy")  \
.withColumnRenamed("watch_acceleration:3d:mean_x","acc_watch_3d_mean_x")  \
.withColumnRenamed("watch_acceleration:3d:mean_y","acc_watch_3d_mean_y")  \
.withColumnRenamed("watch_acceleration:3d:mean_z","acc_watch_3d_mean_z")  \
.withColumnRenamed("watch_acceleration:3d:std_x","acc_watch_3d_std_x")  \
.withColumnRenamed("watch_acceleration:3d:std_y","acc_watch_3d_std_y")  \
.withColumnRenamed("watch_acceleration:3d:std_z","acc_watch_3d_std_z")  \
.withColumnRenamed("watch_acceleration:3d:ro_xy","acc_watch_3d_ro_xy")  \
.withColumnRenamed("watch_acceleration:3d:ro_xz","acc_watch_3d_ro_xz")  \
.withColumnRenamed("watch_acceleration:3d:ro_yz","acc_watch_3d_ro_yz")  \
.withColumnRenamed("watch_heading:mean_cos","acc_watch_head_men_cos")  \
.withColumnRenamed("watch_heading:std_cos","acc_watch_head_std_cos")  \
.withColumnRenamed("watch_heading:mean_sin","acc_watch_head_men_sin")  \
.withColumnRenamed("watch_heading:std_sin","acc_watch_head_std_sin")  \
.withColumnRenamed("watch_heading:entropy_8bins","acc_watch_head_entropy_8bins")  \
.withColumnRenamed("location:num_valid_updates","loc_valid_updates")  \
.withColumnRenamed("location:log_latitude_range","loc_log_latitude_range")  \
.withColumnRenamed("location:log_longitude_range","loc_log_longitude_range")  \
.withColumnRenamed("location:min_altitude","loc_min_altitude")  \
.withColumnRenamed("location:max_altitude","loc_max_altitude")  \
.withColumnRenamed("location:min_speed","loc_min_speed")  \
.withColumnRenamed("location:max_speed","loc_max_speed")  \
.withColumnRenamed("location:best_horizontal_accuracy","loc_best_horizontal_accuracy")  \
.withColumnRenamed("location:best_vertical_accuracy","loc_best_vertical_accuracy")  \
.withColumnRenamed("location:diameter","loc_diameter")  \
.withColumnRenamed("location:log_diameter","loc_log_diameter")  \
.withColumnRenamed("location_quick_features:std_lat","loc_features_std_lat")  \
.withColumnRenamed("location_quick_features:std_long","loc_features_std_long")  \
.withColumnRenamed("location_quick_features:lat_change","loc_features_lat_change")  \
.withColumnRenamed("location_quick_features:long_change","loc_features_log_change")  \
.withColumnRenamed("location_quick_features:mean_abs_lat_deriv","loc_features_mean_abs_lat_deriv")  \
.withColumnRenamed("location_quick_features:mean_abs_long_deriv","loc_features_mean_abs_long_deriv")  \
.withColumnRenamed("lf_measurements:light","lf_meas_light")  \
.withColumnRenamed("lf_measurements:pressure","lf_meas_pressure")  \
.withColumnRenamed("lf_measurements:proximity_cm","lf_meas_proximity_cm")  \
.withColumnRenamed("lf_measurements:proximity","lf_meas_proximity")  \
.withColumnRenamed("lf_measurements:relative_humidity","lf_meas_relative_humidity")  \
.withColumnRenamed("lf_measurements:battery_level","lf_meas_battery_level")  \
.withColumnRenamed("lf_measurements:screen_brightness","lf_meas_screen_brightness")  \
.withColumnRenamed("lf_measurements:temperature_ambient","lf_meas_temperature_ambient")  \
.withColumnRenamed("discrete:time_of_day:between0and6","time_of_day_between0_6")  \
.withColumnRenamed("discrete:time_of_day:between3and9","time_of_day_between3_9")  \
.withColumnRenamed("discrete:time_of_day:between6and12","time_of_day_between6_12")  \
.withColumnRenamed("discrete:time_of_day:between9and15","time_of_day_between9_15")  \
.withColumnRenamed("discrete:time_of_day:between12and18","time_of_day_between12_18")  \
.withColumnRenamed("discrete:time_of_day:between15and21","time_of_day_between15_21")  \
.withColumnRenamed("discrete:time_of_day:between18and24","time_of_day_between18_24")  \
.withColumnRenamed("discrete:time_of_day:between21and3","time_of_day_between21_3")  \
.withColumnRenamed("label:LYING_DOWN","lab_LYING_DOWN")  \
.withColumnRenamed("label:SITTING","lab_SITTING")  \
.withColumnRenamed("label:FIX_walking","lab_FIX_walking")  \
.withColumnRenamed("label:FIX_running","lab_FIX_running")  \
.withColumnRenamed("label:BICYCLING","lab_BICYCLING")  \
.withColumnRenamed("label:SLEEPING","lab_SLEEPING")  \
.withColumnRenamed("label:LAB_WORK","lab_LAB_WORK")  \
.withColumnRenamed("label:IN_CLASS","lab_IN_CLASS")  \
.withColumnRenamed("label:IN_A_MEETING","lab_IN_A_MEETING")  \
.withColumnRenamed("label:LOC_main_workplace","lab_LOC_main_workplace")  \
.withColumnRenamed("label:OR_indoors","lab_OR_indoors")  \
.withColumnRenamed("label:OR_outside","lab_OR_outside")  \
.withColumnRenamed("label:IN_A_CAR","lab_IN_A_CAR")  \
.withColumnRenamed("label:ON_A_BUS","lab_ON_A_BUS")  \
.withColumnRenamed("label:DRIVE_-_I_M_THE_DRIVER","lab_DRIVE_I_M_THE_DRIVER")  \
.withColumnRenamed("label:DRIVE_-_I_M_A_PASSENGER","lab_DRIVE_I_M_A_PASSENGER")  \
.withColumnRenamed("label:LOC_home","lab_LOC_home")  \
.withColumnRenamed("label:FIX_restaurant","lab_FIX_restaurant")  \
.withColumnRenamed("label:PHONE_IN_POCKET","lab_PHONE_IN_POCKET")  \
.withColumnRenamed("label:OR_exercise","lab_OR_exercise")  \
.withColumnRenamed("label:COOKING","lab_COOKING")  \
.withColumnRenamed("label:SHOPPING","lab_SHOPPING")  \
.withColumnRenamed("label:STROLLING","lab_STROLLING")  \
.withColumnRenamed("label:DRINKING__ALCOHOL_","lab_DRINKING__ALCOHOL_")  \
.withColumnRenamed("label:BATHING_-_SHOWER","lab_BATHING_SHOWER")  \
.withColumnRenamed("label:CLEANING","lab_CLEANING")  \
.withColumnRenamed("label:DOING_LAUNDRY","lab_DOING_LAUNDRY")  \
.withColumnRenamed("label:WASHING_DISHES","lab_WASHING_DISHES")  \
.withColumnRenamed("label:WATCHING_TV","lab_WATCHING_TV")  \
.withColumnRenamed("label:SURFING_THE_INTERNET","lab_SURFING_THE_INTERNET")  \
.withColumnRenamed("label:AT_A_PARTY","lab_AT_A_PARTY")  \
.withColumnRenamed("label:AT_A_BAR","lab_AT_A_BAR")  \
.withColumnRenamed("label:LOC_beach","lab_LOC_beach")  \
.withColumnRenamed("label:SINGING","lab_SINGING")  \
.withColumnRenamed("label:TALKING","lab_TALKING")  \
.withColumnRenamed("label:COMPUTER_WORK","lab_COMPUTER_WORK")  \
.withColumnRenamed("label:EATING","lab_EATING")  \
.withColumnRenamed("label:TOILET","lab_TOILET")  \
.withColumnRenamed("label:GROOMING","lab_GROOMING")  \
.withColumnRenamed("label:DRESSING","lab_DRESSING")  \
.withColumnRenamed("label:AT_THE_GYM","lab_AT_THE_GYM")  \
.withColumnRenamed("label:STAIRS_-_GOING_UP","lab_STAIRS_GOING_UP")  \
.withColumnRenamed("label:STAIRS_-_GOING_DOWN","lab_STAIRS_GOING_DOWN")  \
.withColumnRenamed("label:ELEVATOR","lab_ELEVATOR")  \
.withColumnRenamed("label:OR_standing","lab_OR_standing")  \
.withColumnRenamed("label:AT_SCHOOL","lab_AT_SCHOOL")  \
.withColumnRenamed("label:PHONE_IN_HAND","lab_PHONE_IN_HAND")  \
.withColumnRenamed("label:PHONE_IN_BAG","lab_PHONE_IN_BAG")  \
.withColumnRenamed("label:PHONE_ON_TABLE","lab_PHONE_ON_TABLE")  \
.withColumnRenamed("label:WITH_CO-WORKERS","lab_WITH_COWORKERS")  \
.withColumnRenamed("label:WITH_FRIENDS","lab_WITH_FRIENDS")  \


In [8]:
# Original Dataframe dimensions

orig_rows= df.count()
orig_columns= len(df.columns)

print("Dataframe dimensions: ", (orig_rows, orig_columns))
#df.show(n=2, vertical=True)

Dataframe dimensions:  (156426, 147)


# Tranformations: Data types

In [9]:
# Changing unix time format to datetime 
df = df.withColumn("timestamp", from_unixtime('timestamp', "yyyy-MM-dd HH:mm:ss"))

# Setting the correct time_zone = PST ('US/Pacific'), according to http://extrasensory.ucsd.edu/papers/vaizman2017a_pervasiveAcceptedVersion.pdf
df = df.withColumn("timestamp", from_utc_timestamp(df.timestamp, 'PST')) 

# Show values
df.select("timestamp").show(n=10, vertical=True)

-RECORD 0------------------------
 timestamp | 2015-12-11 15:53:10 
-RECORD 1------------------------
 timestamp | 2015-12-11 15:54:41 
-RECORD 2------------------------
 timestamp | 2015-12-11 15:55:35 
-RECORD 3------------------------
 timestamp | 2015-12-11 15:56:09 
-RECORD 4------------------------
 timestamp | 2015-12-11 15:57:09 
-RECORD 5------------------------
 timestamp | 2015-12-11 15:58:41 
-RECORD 6------------------------
 timestamp | 2015-12-11 15:59:41 
-RECORD 7------------------------
 timestamp | 2015-12-11 16:00:36 
-RECORD 8------------------------
 timestamp | 2015-12-11 16:01:09 
-RECORD 9------------------------
 timestamp | 2015-12-11 16:02:41 
only showing top 10 rows



In [10]:
# Change numeric data types

df = df.withColumn("acc_magnitude_mean", df.acc_magnitude_mean.cast(DoubleType()))   \
.withColumn("acc_magnitude_std", df.acc_magnitude_std.cast(DoubleType()))   \
.withColumn("acc_magnitude_value_entropy", df.acc_magnitude_value_entropy.cast(DoubleType()))   \
.withColumn("acc_magnitude_time_entropy", df.acc_magnitude_time_entropy.cast(DoubleType()))   \
.withColumn("acc_spec_spectral_entropy", df.acc_spec_spectral_entropy.cast(DoubleType()))   \
.withColumn("acc_3d_mean_x", df.acc_3d_mean_x.cast(DoubleType()))   \
.withColumn("acc_3d_mean_y", df.acc_3d_mean_y.cast(DoubleType()))   \
.withColumn("acc_3d_mean_z", df.acc_3d_mean_z.cast(DoubleType()))   \
.withColumn("acc_3d_std_x", df.acc_3d_std_x.cast(DoubleType()))   \
.withColumn("acc_3d_std_y", df.acc_3d_std_y.cast(DoubleType()))   \
.withColumn("acc_3d_std_z", df.acc_3d_std_z.cast(DoubleType()))   \
.withColumn("acc_3d_ro_x", df.acc_3d_ro_x.cast(DoubleType()))   \
.withColumn("acc_3d_ro_y", df.acc_3d_ro_y.cast(DoubleType()))   \
.withColumn("acc_3d_ro_z", df.acc_3d_ro_z.cast(DoubleType()))   \
.withColumn("gyro_magnitude_mean", df.gyro_magnitude_mean.cast(DoubleType()))   \
.withColumn("gyro_magnitude_std", df.gyro_magnitude_std.cast(DoubleType()))   \
.withColumn("gyro_magnitude_value_entropy", df.gyro_magnitude_value_entropy.cast(DoubleType()))   \
.withColumn("gyro_magnitude_time_entropy", df.gyro_magnitude_time_entropy.cast(DoubleType()))   \
.withColumn("gyro_spec_spectral_entropy", df.gyro_spec_spectral_entropy.cast(DoubleType()))   \
.withColumn("gyro_3d_mean_x", df.gyro_3d_mean_x.cast(DoubleType()))   \
.withColumn("gyro_3d_mean_y", df.gyro_3d_mean_y.cast(DoubleType()))   \
.withColumn("gyro_3d_mean_z", df.gyro_3d_mean_z.cast(DoubleType()))   \
.withColumn("gyro_3d_std_x", df.gyro_3d_std_x.cast(DoubleType()))   \
.withColumn("gyro_3d_std_y", df.gyro_3d_std_y.cast(DoubleType()))   \
.withColumn("gyro_3d_std_z", df.gyro_3d_std_z.cast(DoubleType()))   \
.withColumn("gyro_3d_ro_xy", df.gyro_3d_ro_xy.cast(DoubleType()))   \
.withColumn("gyro_3d_ro_xz", df.gyro_3d_ro_xz.cast(DoubleType()))   \
.withColumn("gyro_3d_ro_yz", df.gyro_3d_ro_yz.cast(DoubleType()))   \
.withColumn("magnet_magnitude_mean", df.magnet_magnitude_mean.cast(DoubleType()))   \
.withColumn("magnet_magnitude_std", df.magnet_magnitude_std.cast(DoubleType()))   \
.withColumn("magnet_magnitude_value_entropy", df.magnet_magnitude_value_entropy.cast(DoubleType()))   \
.withColumn("magnet_magnitude_time_entropy", df.magnet_magnitude_time_entropy.cast(DoubleType()))   \
.withColumn("magnet_spec_spectral_entropy", df.magnet_spec_spectral_entropy.cast(DoubleType()))   \
.withColumn("magnet_3d_mean_x", df.magnet_3d_mean_x.cast(DoubleType()))   \
.withColumn("magnet_3d_mean_y", df.magnet_3d_mean_y.cast(DoubleType()))   \
.withColumn("magnet_3d_mean_z", df.magnet_3d_mean_z.cast(DoubleType()))   \
.withColumn("magnet_3d_std_x", df.magnet_3d_std_x.cast(DoubleType()))   \
.withColumn("magnet_3d_std_y", df.magnet_3d_std_y.cast(DoubleType()))   \
.withColumn("magnet_3d_std_z", df.magnet_3d_std_z.cast(DoubleType()))   \
.withColumn("magnet_3d_ro_xy", df.magnet_3d_ro_xy.cast(DoubleType()))   \
.withColumn("magnet_3d_ro_xz", df.magnet_3d_ro_xz.cast(DoubleType()))   \
.withColumn("magnet_3d_ro_yz", df.magnet_3d_ro_yz.cast(DoubleType()))   \
.withColumn("acc_watch_magnitude_mean", df.acc_watch_magnitude_mean.cast(DoubleType()))   \
.withColumn("acc_watch_magnitude_std", df.acc_watch_magnitude_std.cast(DoubleType()))   \
.withColumn("acc_watch_magnitude_value_entropy", df.acc_watch_magnitude_value_entropy.cast(DoubleType()))   \
.withColumn("acc_watch_magnitude_time_entropy", df.acc_watch_magnitude_time_entropy.cast(DoubleType()))   \
.withColumn("acc_watch_spec_spectral_entropy", df.acc_watch_spec_spectral_entropy.cast(DoubleType()))   \
.withColumn("acc_watch_3d_mean_x", df.acc_watch_3d_mean_x.cast(DoubleType()))   \
.withColumn("acc_watch_3d_mean_y", df.acc_watch_3d_mean_y.cast(DoubleType()))   \
.withColumn("acc_watch_3d_mean_z", df.acc_watch_3d_mean_z.cast(DoubleType()))   \
.withColumn("acc_watch_3d_std_x", df.acc_watch_3d_std_x.cast(DoubleType()))   \
.withColumn("acc_watch_3d_std_y", df.acc_watch_3d_std_y.cast(DoubleType()))   \
.withColumn("acc_watch_3d_std_z", df.acc_watch_3d_std_z.cast(DoubleType()))   \
.withColumn("acc_watch_3d_ro_xy", df.acc_watch_3d_ro_xy.cast(DoubleType()))   \
.withColumn("acc_watch_3d_ro_xz", df.acc_watch_3d_ro_xz.cast(DoubleType()))   \
.withColumn("acc_watch_3d_ro_yz", df.acc_watch_3d_ro_yz.cast(DoubleType()))   \
.withColumn("acc_watch_head_men_cos", df.acc_watch_head_men_cos.cast(DoubleType()))   \
.withColumn("acc_watch_head_std_cos", df.acc_watch_head_std_cos.cast(DoubleType()))   \
.withColumn("acc_watch_head_men_sin", df.acc_watch_head_men_sin.cast(DoubleType()))   \
.withColumn("acc_watch_head_std_sin", df.acc_watch_head_std_sin.cast(DoubleType()))   \
.withColumn("acc_watch_head_entropy_8bins", df.acc_watch_head_entropy_8bins.cast(DoubleType()))   \
.withColumn("loc_valid_updates", df.loc_valid_updates.cast(IntegerType()))   \
.withColumn("loc_log_latitude_range", df.loc_log_latitude_range.cast(DoubleType()))   \
.withColumn("loc_log_longitude_range", df.loc_log_longitude_range.cast(DoubleType()))   \
.withColumn("loc_min_altitude", df.loc_min_altitude.cast(DoubleType()))   \
.withColumn("loc_max_altitude", df.loc_max_altitude.cast(DoubleType()))   \
.withColumn("loc_min_speed", df.loc_min_speed.cast(DoubleType()))   \
.withColumn("loc_max_speed", df.loc_max_speed.cast(DoubleType()))   \
.withColumn("loc_best_horizontal_accuracy", df.loc_best_horizontal_accuracy.cast(DoubleType()))   \
.withColumn("loc_best_vertical_accuracy", df.loc_best_vertical_accuracy.cast(DoubleType()))   \
.withColumn("loc_diameter", df.loc_diameter.cast(DoubleType()))   \
.withColumn("loc_log_diameter", df.loc_log_diameter.cast(DoubleType()))   \
.withColumn("loc_features_std_lat", df.loc_features_std_lat.cast(DoubleType()))   \
.withColumn("loc_features_std_long", df.loc_features_std_long.cast(DoubleType()))   \
.withColumn("loc_features_lat_change", df.loc_features_lat_change.cast(DoubleType()))   \
.withColumn("loc_features_log_change", df.loc_features_log_change.cast(DoubleType()))   \
.withColumn("loc_features_mean_abs_lat_deriv", df.loc_features_mean_abs_lat_deriv.cast(DoubleType()))   \
.withColumn("loc_features_mean_abs_long_deriv", df.loc_features_mean_abs_long_deriv.cast(DoubleType()))   \
.withColumn("lf_meas_light", df.lf_meas_light.cast(DoubleType()))   \
.withColumn("lf_meas_pressure", df.lf_meas_pressure.cast(DoubleType()))   \
.withColumn("lf_meas_proximity_cm", df.lf_meas_proximity_cm.cast(IntegerType()))   \
.withColumn("lf_meas_proximity", df.lf_meas_proximity.cast(DoubleType()))   \
.withColumn("lf_meas_relative_humidity", df.lf_meas_relative_humidity.cast(DoubleType()))   \
.withColumn("lf_meas_battery_level", df.lf_meas_battery_level.cast(DoubleType()))   \
.withColumn("lf_meas_screen_brightness", df.lf_meas_screen_brightness.cast(DoubleType()))   \
.withColumn("lf_meas_temperature_ambient", df.lf_meas_temperature_ambient.cast(DoubleType()))   \
.withColumn("time_of_day_between0_6", df.time_of_day_between0_6.cast(IntegerType()))   \
.withColumn("time_of_day_between3_9", df.time_of_day_between3_9.cast(IntegerType()))   \
.withColumn("time_of_day_between6_12", df.time_of_day_between6_12.cast(IntegerType()))   \
.withColumn("time_of_day_between9_15", df.time_of_day_between9_15.cast(IntegerType()))   \
.withColumn("time_of_day_between12_18", df.time_of_day_between12_18.cast(IntegerType()))   \
.withColumn("time_of_day_between15_21", df.time_of_day_between15_21.cast(IntegerType()))   \
.withColumn("time_of_day_between18_24", df.time_of_day_between18_24.cast(IntegerType()))   \
.withColumn("time_of_day_between21_3", df.time_of_day_between21_3.cast(IntegerType()))   \
.withColumn("lab_LYING_DOWN", df.lab_LYING_DOWN.cast(IntegerType()))   \
.withColumn("lab_SITTING", df.lab_SITTING.cast(IntegerType()))   \
.withColumn("lab_FIX_walking", df.lab_FIX_walking.cast(IntegerType()))   \
.withColumn("lab_FIX_running", df.lab_FIX_running.cast(IntegerType()))   \
.withColumn("lab_BICYCLING", df.lab_BICYCLING.cast(IntegerType()))   \
.withColumn("lab_SLEEPING", df.lab_SLEEPING.cast(IntegerType()))   \
.withColumn("lab_LAB_WORK", df.lab_LAB_WORK.cast(IntegerType()))   \
.withColumn("lab_IN_CLASS", df.lab_IN_CLASS.cast(IntegerType()))   \
.withColumn("lab_IN_A_MEETING", df.lab_IN_A_MEETING.cast(IntegerType()))   \
.withColumn("lab_LOC_main_workplace", df.lab_LOC_main_workplace.cast(IntegerType()))   \
.withColumn("lab_OR_indoors", df.lab_OR_indoors.cast(IntegerType()))   \
.withColumn("lab_OR_outside", df.lab_OR_outside.cast(IntegerType()))   \
.withColumn("lab_IN_A_CAR", df.lab_IN_A_CAR.cast(IntegerType()))   \
.withColumn("lab_ON_A_BUS", df.lab_ON_A_BUS.cast(IntegerType()))   \
.withColumn("lab_DRIVE_I_M_THE_DRIVER", df.lab_DRIVE_I_M_THE_DRIVER.cast(IntegerType()))   \
.withColumn("lab_DRIVE_I_M_A_PASSENGER", df.lab_DRIVE_I_M_A_PASSENGER.cast(IntegerType()))   \
.withColumn("lab_LOC_home", df.lab_LOC_home.cast(IntegerType()))   \
.withColumn("lab_FIX_restaurant", df.lab_FIX_restaurant.cast(IntegerType()))   \
.withColumn("lab_PHONE_IN_POCKET", df.lab_PHONE_IN_POCKET.cast(IntegerType()))   \
.withColumn("lab_OR_exercise", df.lab_OR_exercise.cast(IntegerType()))   \
.withColumn("lab_COOKING", df.lab_COOKING.cast(IntegerType()))   \
.withColumn("lab_SHOPPING", df.lab_SHOPPING.cast(IntegerType()))   \
.withColumn("lab_STROLLING", df.lab_STROLLING.cast(IntegerType()))   \
.withColumn("lab_DRINKING__ALCOHOL_", df.lab_DRINKING__ALCOHOL_.cast(IntegerType()))   \
.withColumn("lab_BATHING_SHOWER", df.lab_BATHING_SHOWER.cast(IntegerType()))   \
.withColumn("lab_CLEANING", df.lab_CLEANING.cast(IntegerType()))   \
.withColumn("lab_DOING_LAUNDRY", df.lab_DOING_LAUNDRY.cast(IntegerType()))   \
.withColumn("lab_WASHING_DISHES", df.lab_WASHING_DISHES.cast(IntegerType()))   \
.withColumn("lab_WATCHING_TV", df.lab_WATCHING_TV.cast(IntegerType()))   \
.withColumn("lab_SURFING_THE_INTERNET", df.lab_SURFING_THE_INTERNET.cast(IntegerType()))   \
.withColumn("lab_AT_A_PARTY", df.lab_AT_A_PARTY.cast(IntegerType()))   \
.withColumn("lab_AT_A_BAR", df.lab_AT_A_BAR.cast(IntegerType()))   \
.withColumn("lab_LOC_beach", df.lab_LOC_beach.cast(IntegerType()))   \
.withColumn("lab_SINGING", df.lab_SINGING.cast(IntegerType()))   \
.withColumn("lab_TALKING", df.lab_TALKING.cast(IntegerType()))   \
.withColumn("lab_COMPUTER_WORK", df.lab_COMPUTER_WORK.cast(IntegerType()))   \
.withColumn("lab_EATING", df.lab_EATING.cast(IntegerType()))   \
.withColumn("lab_TOILET", df.lab_TOILET.cast(IntegerType()))   \
.withColumn("lab_GROOMING", df.lab_GROOMING.cast(IntegerType()))   \
.withColumn("lab_DRESSING", df.lab_DRESSING.cast(IntegerType()))   \
.withColumn("lab_AT_THE_GYM", df.lab_AT_THE_GYM.cast(IntegerType()))   \
.withColumn("lab_STAIRS_GOING_UP", df.lab_STAIRS_GOING_UP.cast(IntegerType()))   \
.withColumn("lab_STAIRS_GOING_DOWN", df.lab_STAIRS_GOING_DOWN.cast(IntegerType()))   \
.withColumn("lab_ELEVATOR", df.lab_ELEVATOR.cast(IntegerType()))   \
.withColumn("lab_OR_standing", df.lab_OR_standing.cast(IntegerType()))   \
.withColumn("lab_AT_SCHOOL", df.lab_AT_SCHOOL.cast(IntegerType()))   \
.withColumn("lab_PHONE_IN_HAND", df.lab_PHONE_IN_HAND.cast(IntegerType()))   \
.withColumn("lab_PHONE_IN_BAG", df.lab_PHONE_IN_BAG.cast(IntegerType()))   \
.withColumn("lab_PHONE_ON_TABLE", df.lab_PHONE_ON_TABLE.cast(IntegerType()))   \
.withColumn("lab_WITH_COWORKERS", df.lab_WITH_COWORKERS.cast(IntegerType()))   \
.withColumn("lab_WITH_FRIENDS", df.lab_WITH_FRIENDS.cast(IntegerType()))   \


In [ ]:
df.printSchema()

In [11]:
# Show values
df.show(n=2, vertical=True, truncate=100)

-RECORD 0-----------------------------------------------------------------
 timestamp                         | 2015-12-11 15:53:10                  
 acc_magnitude_mean                | 1.005899                             
 acc_magnitude_std                 | 0.013165                             
 acc_magnitude_value_entropy       | 1.185242                             
 acc_magnitude_time_entropy        | 6.684527                             
 acc_spec_spectral_entropy         | 0.431292                             
 acc_3d_mean_x                     | -0.002976                            
 acc_3d_mean_y                     | 0.015498                             
 acc_3d_mean_z                     | 1.005656                             
 acc_3d_std_x                      | 0.011572                             
 acc_3d_std_y                      | 0.010292                             
 acc_3d_std_z                      | 0.01317                              
 acc_3d_ro_x             

# Transformations: Null values

In [ ]:
# Evaluate null values for each column
# for col in df.columns:
#   print(col, "\t", "Nulls: ", df.filter(df[col].isNull()).count())


In [12]:
# Replace null values with 0 for all label columns

label_columns = []

for col in df.columns:
  if('lab' in col):
    label_columns.append(col)

df = df.na.fill(value=0,subset=label_columns)

In [ ]:
# Remove rows with null values on all columns 
df = df.na.drop("all")

print("Removed rows: ", (orig_rows - df.count()))

In [13]:
# Remove rows with any null value on columns for high-frequency motion-reactive sensors: accelerometer, gyroscope
# The sensors magnetometer and watch accelerometer are also motion-reactive sensors, although these columns contain more than 20% of null values. 
# Another null-handling strategy will be use for these columns

motion_sensor_columns = [
"acc_magnitude_mean",
"acc_magnitude_std",
"acc_magnitude_value_entropy",
"acc_magnitude_time_entropy",
"acc_spec_spectral_entropy",
"acc_3d_mean_x",
"acc_3d_mean_y",
"acc_3d_mean_z",
"acc_3d_std_x",
"acc_3d_std_y",
"acc_3d_std_z",
"acc_3d_ro_x",
"acc_3d_ro_y",
"acc_3d_ro_z",
"gyro_magnitude_mean",
"gyro_magnitude_std",
"gyro_magnitude_value_entropy",
"gyro_magnitude_time_entropy",
"gyro_spec_spectral_entropy",
"gyro_3d_mean_x",
"gyro_3d_mean_y",
"gyro_3d_mean_z",
"gyro_3d_std_x",
"gyro_3d_std_y",
"gyro_3d_std_z",
"gyro_3d_ro_xy",
"gyro_3d_ro_xz",
"gyro_3d_ro_yz"]

df = df.na.drop(subset= motion_sensor_columns)

#print("Removed rows: ", (orig_rows - df.count()))
# print("Removed ", ((orig_rows - df.count())*100)/orig_rows, "% of total rows")

In [14]:
# Define columns for sensor values excluding those columns for high-frequency motion-reactive sensors: accelerometer, gyroscope

sensor_columns = [f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
double_columns = [x for x in sensor_columns if x not in motion_sensor_columns]
print("number of double columns: ", len(double_columns))
double_columns

number of double columns:  56


['magnet_magnitude_mean',
 'magnet_magnitude_std',
 'magnet_magnitude_value_entropy',
 'magnet_magnitude_time_entropy',
 'magnet_spec_spectral_entropy',
 'magnet_3d_mean_x',
 'magnet_3d_mean_y',
 'magnet_3d_mean_z',
 'magnet_3d_std_x',
 'magnet_3d_std_y',
 'magnet_3d_std_z',
 'magnet_3d_ro_xy',
 'magnet_3d_ro_xz',
 'magnet_3d_ro_yz',
 'acc_watch_magnitude_mean',
 'acc_watch_magnitude_std',
 'acc_watch_magnitude_value_entropy',
 'acc_watch_magnitude_time_entropy',
 'acc_watch_spec_spectral_entropy',
 'acc_watch_3d_mean_x',
 'acc_watch_3d_mean_y',
 'acc_watch_3d_mean_z',
 'acc_watch_3d_std_x',
 'acc_watch_3d_std_y',
 'acc_watch_3d_std_z',
 'acc_watch_3d_ro_xy',
 'acc_watch_3d_ro_xz',
 'acc_watch_3d_ro_yz',
 'acc_watch_head_men_cos',
 'acc_watch_head_std_cos',
 'acc_watch_head_men_sin',
 'acc_watch_head_std_sin',
 'acc_watch_head_entropy_8bins',
 'loc_log_latitude_range',
 'loc_log_longitude_range',
 'loc_min_altitude',
 'loc_max_altitude',
 'loc_min_speed',
 'loc_max_speed',
 'loc_best_h

In [15]:
# Remove columns with more than 50% null values

# Define 50% threshold
nan_percent=0.5
threshold = df.count() * nan_percent

print("Max number of null rows: ", threshold)

Max number of null rows:  73990.5


In [16]:
# Define columns with more than 50% null values

rem_columns = []

for col in double_columns:
  if(df.filter(df[col].isNull()).count() >= threshold):
    rem_columns.append(col) 

In [17]:
# Remove columns with more than 50% null values

df = df.drop(*rem_columns)

sensor_columns = [f.name for f in df.schema.fields if isinstance(f.dataType, DoubleType)]
double_columns = [x for x in sensor_columns if x not in motion_sensor_columns]

print("Removed columns: ", rem_columns)

Removed columns:  ['acc_watch_head_men_cos', 'acc_watch_head_std_cos', 'acc_watch_head_men_sin', 'acc_watch_head_std_sin', 'acc_watch_head_entropy_8bins', 'loc_min_speed', 'loc_max_speed', 'lf_meas_light', 'lf_meas_pressure', 'lf_meas_relative_humidity', 'lf_meas_temperature_ambient']


In [18]:
# Tranformed Dataframe dimensions

trans_rows= df.count()
trans_columns= len(df.columns)

print("Transformed Dataframe dimensions: ", (df.count(), len(df.columns)))
print("Rows/Dimensions reduced: ", (orig_rows-trans_rows, orig_columns-trans_columns))

Transformed Dataframe dimensions:  (147981, 136)
Rows/Dimensions reduced:  (8445, 11)


# Transformations : Replace values

In [ ]:
# Pending Null values in sensor columns
#df_select = df.select(*double_columns)
#pddf = df_select.toPandas()

#pddf.isna().sum()

In [43]:
import pandas as pd
from scipy import stats

df_select = df.select(*double_columns)
pddf = df_select.toPandas()

# In order to replace null values with mean value for each sensor (double type) column, the normality for each one is validated
# using D’Agostino and Pearson’s test that combines skew and kurtosis to produce an omnibus test of normality.
# Tests the null hypothesis that a sample comes from a normal distribution. 
# H0: Fs(x) is equal to Ft(x) for all x --> variable normally distributed
# HA: Fs(x) is not equal to Ft(x) for at least one x  --> variable not normally distributed

p_list = []

for col in double_columns:
    col_nonull = pddf[col].loc[pddf[col].notnull()]
    k2, p = stats.normaltest(col_nonull)
    pair = {col:p}
    p_list.append(pair)
    print (pair)

# If p<0.5 Not normally ditributed (Null hypothesis rejected)

{'magnet_magnitude_mean': 0.0}
{'magnet_magnitude_std': 0.0}
{'magnet_magnitude_value_entropy': 0.0}
{'magnet_magnitude_time_entropy': 0.0}
{'magnet_spec_spectral_entropy': 0.0}
{'magnet_3d_mean_x': 0.0}
{'magnet_3d_mean_y': 0.0}
{'magnet_3d_mean_z': 0.0}
{'magnet_3d_std_x': 0.0}
{'magnet_3d_std_y': 0.0}
{'magnet_3d_std_z': 0.0}
{'magnet_3d_ro_xy': 0.0}
{'magnet_3d_ro_xz': 0.0}
{'magnet_3d_ro_yz': 0.0}
{'acc_watch_magnitude_mean': 0.0}
{'acc_watch_magnitude_std': 0.0}
{'acc_watch_magnitude_value_entropy': 0.0}
{'acc_watch_magnitude_time_entropy': 0.0}
{'acc_watch_spec_spectral_entropy': 0.0}
{'acc_watch_3d_mean_x': 5.79832738219175e-166}
{'acc_watch_3d_mean_y': 0.0}
{'acc_watch_3d_mean_z': 0.0}
{'acc_watch_3d_std_x': 0.0}
{'acc_watch_3d_std_y': 0.0}
{'acc_watch_3d_std_z': 0.0}
{'acc_watch_3d_ro_xy': 8.015002223109158e-52}
{'acc_watch_3d_ro_xz': 9.510778565961642e-62}
{'acc_watch_3d_ro_yz': 1.6505325202518306e-17}
{'loc_log_latitude_range': 0.0}
{'loc_log_longitude_range': 0.0}
{'loc_mi

In [32]:
# Calculate average value for each user and each double_columns (sensor columns with null values)

exprs1 = [mean(x).alias("{}_avg".format(x)) for x in double_columns]
averages = df.groupBy("user_id").agg(*exprs1)

# print("averages dimensions: ", (averages.count(), len(averages.columns)))
# averages dimensions:  (29, 91)

In [33]:
# Join calculated averages with df on user_id
df_with_avgs = df.join(averages, on="user_id")

# Apply coalesce to columns in order to replace the null values by the mean value calculated
exprs2 = ["{}_avg".format(x) for x in double_columns]
for col in double_columns:
    df = df_with_avgs.withColumn(col, coalesce(col, *exprs2))
    
# Drop avg columns and preserve only those imputed

In [34]:
drop_avg_columns = []

for col in df.columns:
  if('_avg' in col):
    drop_avg_columns.append(col)
    
df = df.drop(*drop_avg_columns)

print("Removed columns: ", drop_avg_columns)

Removed columns:  ['magnet_magnitude_mean_avg', 'magnet_magnitude_std_avg', 'magnet_magnitude_value_entropy_avg', 'magnet_magnitude_time_entropy_avg', 'magnet_spec_spectral_entropy_avg', 'magnet_3d_mean_x_avg', 'magnet_3d_mean_y_avg', 'magnet_3d_mean_z_avg', 'magnet_3d_std_x_avg', 'magnet_3d_std_y_avg', 'magnet_3d_std_z_avg', 'magnet_3d_ro_xy_avg', 'magnet_3d_ro_xz_avg', 'magnet_3d_ro_yz_avg', 'acc_watch_magnitude_mean_avg', 'acc_watch_magnitude_std_avg', 'acc_watch_magnitude_value_entropy_avg', 'acc_watch_magnitude_time_entropy_avg', 'acc_watch_spec_spectral_entropy_avg', 'acc_watch_3d_mean_x_avg', 'acc_watch_3d_mean_y_avg', 'acc_watch_3d_mean_z_avg', 'acc_watch_3d_std_x_avg', 'acc_watch_3d_std_y_avg', 'acc_watch_3d_std_z_avg', 'acc_watch_3d_ro_xy_avg', 'acc_watch_3d_ro_xz_avg', 'acc_watch_3d_ro_yz_avg', 'loc_log_latitude_range_avg', 'loc_log_longitude_range_avg', 'loc_min_altitude_avg', 'loc_max_altitude_avg', 'loc_best_horizontal_accuracy_avg', 'loc_best_vertical_accuracy_avg', 'loc

In [35]:
# Tranformed Dataframe dimensions

trans_rows= df.count()
trans_columns= len(df.columns)

print("Transformed Dataframe dimensions: ", (df.count(), len(df.columns)))
print("Rows/Dimensions reduced: ", (orig_rows-trans_rows, orig_columns-trans_columns))

Transformed Dataframe dimensions:  (147981, 136)
Rows/Dimensions reduced:  (8445, 11)


# Create label column

In [44]:
# New column to mark labels related with geographical displacement/shift 

from pyspark.sql.functions import *

df = df.withColumn('lab_new_geographical_displac',
    when((col("lab_FIX_walking") == 1), 1)\
    .when((col("lab_FIX_running") == 1), 1)\
    .when((col("lab_BICYCLING") == 1), 1)\
    .when((col("lab_IN_A_CAR") == 1), 1)\
    .when((col("lab_ON_A_BUS") == 1), 1)\
    .when((col("lab_DRIVE_I_M_THE_DRIVER") == 1), 1)\
    .when((col("lab_DRIVE_I_M_A_PASSENGER") == 1), 1)\
    .when((col("lab_STROLLING") == 1), 1)\
    .when((col("lab_AT_THE_GYM") == 1), 1)\
    .when((col("lab_STAIRS_GOING_UP") == 1), 1)\
    .when((col("lab_STAIRS_GOING_DOWN") == 1), 1)\
    .otherwise(0)
)

In [45]:
# New column to mark labels related with geographical location

from pyspark.sql.functions import *

df = df.withColumn('lab_new_geographical_locat',
    when((col("lab_LOC_main_workplace") == 1), 1)\
    .when((col("lab_LOC_home") == 1), 1)\
    .when((col("lab_LOC_beach") == 1), 1)\
    .when((col("lab_LAB_WORK") == 1), 1)\
    .when((col("lab_IN_CLASS") == 1), 1)\
    .when((col("lab_IN_A_MEETING") == 1), 1)\
    .when((col("lab_OR_indoors") == 1), 1)\
    .when((col("lab_OR_outside") == 1), 1)\
    .when((col("lab_FIX_restaurant") == 1), 1)\
    .when((col("lab_AT_A_PARTY") == 1), 1)\
    .when((col("lab_AT_A_BAR") == 1), 1)\
    .when((col("lab_AT_SCHOOL") == 1), 1)\
    .otherwise(0)
)

In [46]:
drop_lab_columns = []

for col in df.columns:
  if('lab_' in col) and ('lab_new_geographical_displac' not in col) and ('lab_new_geographical_locat' not in col):
    drop_lab_columns.append(col)
    
df = df.drop(*drop_lab_columns)

print("Removed columns: ", drop_lab_columns)

Removed columns:  ['lab_LYING_DOWN', 'lab_SITTING', 'lab_FIX_walking', 'lab_FIX_running', 'lab_BICYCLING', 'lab_SLEEPING', 'lab_LAB_WORK', 'lab_IN_CLASS', 'lab_IN_A_MEETING', 'lab_LOC_main_workplace', 'lab_OR_indoors', 'lab_OR_outside', 'lab_IN_A_CAR', 'lab_ON_A_BUS', 'lab_DRIVE_I_M_THE_DRIVER', 'lab_DRIVE_I_M_A_PASSENGER', 'lab_LOC_home', 'lab_FIX_restaurant', 'lab_PHONE_IN_POCKET', 'lab_OR_exercise', 'lab_COOKING', 'lab_SHOPPING', 'lab_STROLLING', 'lab_DRINKING__ALCOHOL_', 'lab_BATHING_SHOWER', 'lab_CLEANING', 'lab_DOING_LAUNDRY', 'lab_WASHING_DISHES', 'lab_WATCHING_TV', 'lab_SURFING_THE_INTERNET', 'lab_AT_A_PARTY', 'lab_AT_A_BAR', 'lab_LOC_beach', 'lab_SINGING', 'lab_TALKING', 'lab_COMPUTER_WORK', 'lab_EATING', 'lab_TOILET', 'lab_GROOMING', 'lab_DRESSING', 'lab_AT_THE_GYM', 'lab_STAIRS_GOING_UP', 'lab_STAIRS_GOING_DOWN', 'lab_ELEVATOR', 'lab_OR_standing', 'lab_AT_SCHOOL', 'lab_PHONE_IN_HAND', 'lab_PHONE_IN_BAG', 'lab_PHONE_ON_TABLE', 'lab_WITH_COWORKERS', 'lab_WITH_FRIENDS']


In [47]:
# Final Dataframe dimensions

trans_rows= df.count()
trans_columns= len(df.columns)

print("Transformed Dataframe dimensions: ", (df.count(), len(df.columns)))
print("Rows/Dimensions reduced: ", (orig_rows-trans_rows, orig_columns-trans_columns))

Transformed Dataframe dimensions:  (147981, 87)
Rows/Dimensions reduced:  (8445, 60)


# Persist output in Postgres DB

In [48]:
df.printSchema()


root
 |-- user_id: string (nullable = false)
 |-- timestamp: timestamp (nullable = true)
 |-- acc_magnitude_mean: double (nullable = true)
 |-- acc_magnitude_std: double (nullable = true)
 |-- acc_magnitude_value_entropy: double (nullable = true)
 |-- acc_magnitude_time_entropy: double (nullable = true)
 |-- acc_spec_spectral_entropy: double (nullable = true)
 |-- acc_3d_mean_x: double (nullable = true)
 |-- acc_3d_mean_y: double (nullable = true)
 |-- acc_3d_mean_z: double (nullable = true)
 |-- acc_3d_std_x: double (nullable = true)
 |-- acc_3d_std_y: double (nullable = true)
 |-- acc_3d_std_z: double (nullable = true)
 |-- acc_3d_ro_x: double (nullable = true)
 |-- acc_3d_ro_y: double (nullable = true)
 |-- acc_3d_ro_z: double (nullable = true)
 |-- gyro_magnitude_mean: double (nullable = true)
 |-- gyro_magnitude_std: double (nullable = true)
 |-- gyro_magnitude_value_entropy: double (nullable = true)
 |-- gyro_magnitude_time_entropy: double (nullable = true)
 |-- gyro_spec_spectra

In [ ]:
# Persist final dataframe into a table in the DB
df \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/ha_db") \
    .option("dbtable", "human_activity.ha_dataset") \
    .option("user", "ha_user") \
    .option("password", "hnm4/4c71v1tY") \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [ ]:
spark.stop()